<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [17]:
%matplotlib inline

# ----------------- Classics -------------------- #
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# ---------------- Pandas settings --------------- #
# Removes rows and columns truncation of '...'
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


# ------------------- Python libs ---------------- #
import os
import re

# -------------------- NLP ----------------------- #
import spacy
nlp = spacy.load('en_core_web_lg')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import pyLDAvis.gensim

### Load the `reviews`

In [2]:
yelp = pd.read_json('./data/review_sample.json', lines=True)

In [3]:
def wrangle(df):
    df = df.copy()
    df['text'] = df['text'].str.lower().str.strip()
    df['text'] = df['text'].str.replace(r'[^a-zA-Z ^0-9]', '')
    return df

yelp = wrangle(yelp)

In [4]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,beware fake fake fakewe also own a small busin...,10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,came here for lunch togo service was quick sta...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,ive been to vegas dozens of times and had neve...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,we went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,35 to 4 starsnot bad for the price 1299 for lu...,5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [5]:
STOPWORDS = set(STOPWORDS).union(set([' ', '  ', '   ']))

def tokenize(text):
    return [token for token in simple_preprocess(text, deacc=True, min_len=4, max_len=20) if token not in STOPWORDS]

In [6]:
yelp['tokenized'] = yelp['text'].apply(tokenize)
yelp['tokenized'].head()

0    [beware, fake, fake, fakewe, small, business, ...
1    [came, lunch, togo, service, quick, staff, fri...
2    [vegas, dozens, times, stepped, foot, circus, ...
3    [went, night, closed, street, party, best, act...
4    [starsnot, price, lunch, seniors, eating, food...
Name: tokenized, dtype: object

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [7]:
vect = TfidfVectorizer(tokenizer=tokenize)

dtm = vect.fit_transform(yelp['text'])
dtm_df = pd.DataFrame(dtm.todense(),columns=vect.get_feature_names())
dtm_df.head()

,aaaahhhs,aaasssk,aabs,aamco,aand,aaron,aback,abandoned,abby,abdc,abdominal,abend,aber,aberration,abgeht,abhorrent,abiance,abide,abiding,abigail,abilities,ability,abit,ablation,able,abmormal,abnormal,abnormally,aboard,abordable,abordablesa,abound,abouti,aboutonion,aboutour,aboutthe,aboutthen,aboveaverage,aboveif,abraham,abrasive,abreast,abroad,abrupt,abruptly,abruzzo,abscess,absence,absent,absinthe,absintherinsed,absolument,absolute,absolutely,absolutelythe,absolutley,absolutly,absolving,absorb,absorbed,absorbs,abspilates,abstains,abstecher,absurd,absurdim,absurdly,abundance,abundant,abundantwas,aburi,aburiya,aburri,abuse,abusive,abut,abutment,abutments,abyss,abyssinian,academic,academy,acadia,acai,acapella,acapulco,acbad,acceleration,accent,accented,accentuate,accept,acceptable,acceptableat,acceptableill,acceptableneedless,acceptablethe,accepted,accepting,accepts,...,yummylooking,yummymore,yummysome,yummythe,yummythey,yummythis,yummyyummy,yumskip,yumthe,yumthey,yumthis,yumyumyum,yung,yuphas,yuppie,yuppy,yupwe,yuzu,yvette,ywiec,yyzyul,zabas,zach,zachs,zack,zagats,zales,zambroski,zane,zanet,zanzibar,zara,zaru,zcafe,zealand,zebraprint,zedd,zenit,zeppes,zerdin,zero,zeroi,zerorez,zerothe,zest,zesto,zesty,zeus,zhajiang,zhou,ziggys,zillion,zimbabwe,zimbawean,zimbrick,zinburgers,zinc,zinfandel,zing,zinho,zion,zipline,ziplining,zipping,zipps,ziti,zocalo,zoes,zoey,zoltons,zomato,zombie,zombieand,zombies,zone,zoned,zonefor,zoodles,zoom,zoomers,zooming,zoomline,zorro,zroif,ztejas,zthis,zucchini,zucchiniolive,zuccotto,zuma,zumanity,zumba,zuni,zupas,zuzana,zuzu,zxkxko,zyrtec,zzaplon,zzzzzzzzzmy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [8]:
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm_df)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [9]:
bad_fake_review_text = ["The food sucks at this place. I have been avoiding this place for long time, but gave it a try, what can go wrong right? Went with my family, was not disappointed by their horrible food, and customer service."]
bad_text_transformed = vect.transform(bad_fake_review_text)
_, similar_review_indices = nn.kneighbors(bad_text_transformed.todense())

In [10]:
similar_reviews = [yelp.text[t] for t in similar_review_indices]

for review in similar_reviews:
    print(review)

6204                                                     
6311                                                     
9035    i am very disappointed with this place their c...
3230    this place just sucks food was terrible servic...
1922    horrible customer service and management unwil...
Name: text, dtype: object


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

### TfidfVectorizer

In [19]:
vect = TfidfVectorizer(stop_words='english')

svd = TruncatedSVD(algorithm='randomized')

sgdc = SGDClassifier(average=True)

pipe1 = Pipeline([('vect', vect), ('svd', svd), ('sgdc', sgdc)])

parameters1 = { 
    'svd__n_iter': (10, 15, 20),
    'svd__n_components': (100, 300, 400, 500, 600, 700, 800, 900, 1000),
    'sgdc__class_weight': ('balanced',),
    'sgdc__loss': ('hinge', 'log'),
    'sgdc__alpha': (0.0007,),
    'sgdc__tol' : (.0015,), 
    'sgdc__average' : (True, False),
    'sgdc__penalty' : ('l2', 'l1', 'elasticnet')
}

# parameters1 = { 
#     'svd__n_iter': (10, 15, 20),
#     'svd__n_components': (200, 400, 600, 800, 1000),
#     'sgdc__class_weight': ('balanced',),
#     'sgdc__loss': ('hinge', 'log'),
#     'sgdc__alpha': (0.0007,),
#     'sgdc__tol' : (.0015,)                      
# }

clf1 = RandomizedSearchCV(pipe1, parameters1, cv=5, n_jobs=-1, verbose=10)
clf1.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 14.2min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 15.1min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'svd__n_iter': (10, 15, 20), 'svd__n_components': (100, 300, 400, 500, 600, 700, 800, 900, 1000), 'sgdc__class_weight': ('balanced',), 'sgdc__loss': ('hinge', 'log'), 'sgdc__alpha': (0.0007,), 'sgdc__tol': (0.0015,), 'sgdc__average': (True, False), 'sgdc__penalty': ('l2', 'l1', 'elasticnet')},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn',

In [21]:
print(f'Best score out of randomized search using TF/IDF vectorizer is {clf1.best_score_}')

Best score out of randomized search using TF/IDF vectorizer is 0.6028


In [22]:
clf1.best_params_

{'svd__n_iter': 15,
 'svd__n_components': 600,
 'sgdc__tol': 0.0015,
 'sgdc__penalty': 'l2',
 'sgdc__loss': 'log',
 'sgdc__class_weight': 'balanced',
 'sgdc__average': False,
 'sgdc__alpha': 0.0007}

### Stretch Goal - comparing `CountVectorizer`

In [25]:
vect = CountVectorizer()
svd = TruncatedSVD(algorithm='randomized')
sgdc = SGDClassifier(average=True)

pipe2 = Pipeline([('vect', vect), ('svd', svd), ('sgdc', sgdc)])

parameters2 = { 
    'svd__n_iter': (15,),
    'svd__n_components': (600, 700, 800, 900, 1000),
    'sgdc__class_weight': ('balanced',),
    'sgdc__loss': ('log',),
    'sgdc__alpha': (0.0007,),
    'sgdc__tol' : (.0015,),
    'sgdc__penalty' : ('l2',)
}

clf2 = RandomizedSearchCV(pipe2, parameters2, cv=5, n_jobs=-1, verbose=10)
clf2.fit(yelp['text'], yelp['stars'])

Fitting 5 folds for each of 5 candidates, totalling 25 fits


/home/asura/miniconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/sklearn/model_selection/_search.py:271: UserWarning: The total space of parameters 5 is smaller than n_iter=10. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  6.1min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done  21 out of  25 | elapsed: 14.0min remaining:  2.7min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 16.1min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=-1,
          param_distributions={'svd__n_iter': (15,), 'svd__n_components': (600, 700, 800, 900, 1000), 'sgdc__class_weight': ('balanced',), 'sgdc__loss': ('log',), 'sgdc__alpha': (0.0007,), 'sgdc__tol': (0.0015,), 'sgdc__penalty': ('l2',)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=10)

In [26]:
print(f'Best score out of randomized search using Count vectorizer is {clf2.best_score_}')

Best score out of randomized search using Count vectorizer is 0.6102


In [27]:
clf2.best_params_

{'svd__n_iter': 15,
 'svd__n_components': 600,
 'sgdc__tol': 0.0015,
 'sgdc__penalty': 'l2',
 'sgdc__loss': 'log',
 'sgdc__class_weight': 'balanced',
 'sgdc__alpha': 0.0007}

From the randomized search, we can see that best score of `TF/IDF vectorizer` is better than the `CountVectorizer`, as `TF/IDF` reflects how certain words are important to a review in collection of reviews, as such it improves the information retrieval process. 

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

### Stretch Goal - use generator object

In [28]:
def get_reviews(df, token_col):
    """
    Generator function that takes a dataframe with a tokenized column and yields a tokenized 
    row for processing
    
    Parameters:
    -----------
    df: dataframe
    taken_col: name of the column of tokenized text
    
    Yields:
    -------
    review_token: one record of tokenized review
    """
    assert token_col in df.columns, f"{token_col} does not exist!"
    
    for index, row in df.iterrows():
        review_token = row[token_col]
        yield review_token

### 1. Estimate a LDA topic model of the review text

#### Learn the vocubalary of the yelp data:

In [29]:
id2word = corpora.Dictionary(get_reviews(yelp, 'tokenized'))

#### Create a bag of words representation of the entire corpus

In [30]:
corpus = [id2word.doc2bow(review) for review in get_reviews(yelp, 'tokenized')]

#### 1. Your LDA model should be ready for estimation: 

In [31]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

In [32]:
def print_lda_topics(lda):
    words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]
    topics = [' '.join(t[0:5]) for t in words]
    for id, t in enumerate(topics): 
        print(f"------ Topic {id} ------")
        print(t, end="\n\n")
print_lda_topics(lda)

------ Topic 0 ------
food service great good place

------ Topic 1 ------
place food like service great

------ Topic 2 ------
place good great food service

------ Topic 3 ------
great time place food good

------ Topic 4 ------
place food great good time

------ Topic 5 ------
food good great place service

------ Topic 6 ------
food good time service place

------ Topic 7 ------
good place food like great

------ Topic 8 ------
good food like place great

------ Topic 9 ------
good place great food like



### 2. Create 1-2 visualizations of the results

In [35]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda, corpus, id2word)

/home/asura/miniconda3/envs/U4-S1-NLP/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.001276 -0.003953       1        1  10.799973
9      0.000113  0.000541       2        1  10.448765
6     -0.000364  0.004838       3        1  10.216578
4     -0.000543  0.000947       4        1   9.911463
5      0.000862 -0.003765       5        1   9.870988
0     -0.002429  0.002544       6        1   9.834400
8      0.002687  0.000448       7        1   9.802177
7      0.005216  0.000079       8        1   9.801064
3     -0.005026 -0.001082       9        1   9.787970
2      0.000761 -0.000596      10        1   9.526616, topic_info=      Category         Freq                 Term        Total  loglift  \
194    Default  3957.000000                great  3957.000000  30.0000   
32     Default  3155.000000                 time  3155.000000  29.0000   
38     Default  4216.000000                 good  4216.000000  28.0000   
125    Default  4384.000000                place  4384.000000  27.0000   
109    Default  1380.000000                 went  1380.000000  26.0000   
688    Default  1531.000000                 love  1531.000000  25.0000   
43     Default  3269.000000              service  3269.000000  24.0000   
186    Default  1787.000000                 dont  1787.000000  23.0000   
133    Default  4478.000000                 food  4478.000000  22.0000   
170    Default  1091.000000            recommend  1091.000000  21.0000   
17     Default  3204.000000                 like  3204.000000  20.0000   
44     Default  1436.000000                staff  1436.000000  19.0000   
386    Default   863.000000                 took   863.000000  18.0000   
332    Default  1335.000000           definitely  1335.000000  17.0000   
114    Default  1687.000000                 best  1687.000000  16.0000   
346    Default   865.000000                fresh   865.000000  15.0000   
123    Default  1305.000000               people  1305.000000  14.0000   
35     Default  1396.000000                 came  1396.000000  13.0000   
326    Default  1361.000000                 come  1361.000000  12.0000   
151    Default   824.000000                think   824.000000  11.0000   
37     Default  1258.000000             friendly  1258.000000  10.0000   
85     Default  1620.000000                 nice  1620.000000   9.0000   
34     Default   904.000000                 wait   904.000000   8.0000   
369    Default   731.000000                pizza   731.000000   7.0000   
143    Default   988.000000               pretty   988.000000   6.0000   
397    Default  1065.000000               better  1065.000000   5.0000   
616    Default  1499.000000                didnt  1499.000000   4.0000   
281    Default   386.000000                 shop   386.000000   3.0000   
103    Default   788.000000                times   788.000000   2.0000   
227    Default   534.000000                 meat   534.000000   1.0000   
34140   Topic1     1.562226           imcomplete     3.038915   1.5602   
23603   Topic1     0.704474             everrrrr     1.590319   1.4114   
30485   Topic1     1.032335                momma     3.027263   1.1498   
25695   Topic1     0.531417               carted     1.588107   1.1309   
33944   Topic1     1.001844                 dahl     3.027248   1.1198   
33273   Topic1     0.977985              conchas     3.035531   1.0930   
34500   Topic1     0.963859                dolci     3.030131   1.0802   
32048   Topic1     0.954766              engages     3.035704   1.0689   
23581   Topic1     0.495367          smiledouble     1.588639   1.0603   
31385   Topic1     0.890939      waitresscashier     3.023913   1.0036   
32031   Topic1     0.884236              shipper     3.029320   0.9943   
27295   Topic1     0.457575                sauga     1.589502   0.9804   
30487   Topic1     0.866599               spotmy     3.026033   0.9752   
33337   Topic1     2.040996                diana     

### 3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

Looking at the visualizations of the reviews, we can see that most yelpers leave on average positive review, whereby mostly talking about how great the place, food and it's customer service; evident by keywords like: `good`, `place`, `service` `food` etc. 

We can also on other hand remove overwhelmingly positive words like `good`, `great`, etc., and extract more information about Why the food was great, or service was great ? out of the reviews. If we do that, than we would also have to remove overwhelming negative words like `beware`, `bad`, `horrible` from the other end of spectrum, so more rounds of filtering with custom `STOP WORDS` would help narrow down the topics.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)